In [9]:
import pandas as pd
from sqlalchemy import create_engine

# --- Step 1: Connect to Your PostgreSQL Database ---
# IMPORTANT: Fill in your credentials from Supabase here.
# Be careful not to share your password.

db_credentials = {
    'user': 'postgres',      # e.g., 'postgres'
    'password': 'PKBcUUFzb4AlIxQX',
    'host': 'db.farltqwthsfqfmzuzusi.supabase.co',      # e.g., 'aws-0-us-west-1.pooler.supabase.com'
    'port': '5432',      # e.g., '5432'
    'database': 'postgres'  # e.g., 'postgres'
}

# Create the database connection engine
engine = create_engine(f"postgresql+psycopg2://{db_credentials['user']}:{db_credentials['password']}@{db_credentials['host']}:{db_credentials['port']}/{db_credentials['database']}")

print("Successfully created database engine.")


# --- Step 2: Load and Clean Data in Python ---
df = pd.read_csv('marketing_campaign.csv', sep=';')

# Handle missing income values by filling with the median
df['Income'] = df['Income'].fillna(df['Income'].median())

# Convert customer sign-up date to a proper datetime format
# This new line will handle the mixed formats
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='mixed')

# Engineer a 'TotalSpending' feature for easier analysis
mnt_cols = [col for col in df.columns if 'Mnt' in col]
df['TotalSpending'] = df[mnt_cols].sum(axis=1)

print("Data loaded and cleaned. 'TotalSpending' feature created.")


# --- Step 3: Upload the Cleaned Data to PostgreSQL ---
# This will create a new table called 'customer_data' in your Supabase database.
df.to_sql('customer_data', engine, if_exists='replace', index=False)

print("Successfully uploaded cleaned data to PostgreSQL database.")


# --- Step 4: Run an Advanced SQL Query ---
# Goal: Find the top 5 customers by total spending within each education level.
# This requires a Common Table Expression (CTE) and a Window Function (RANK).

advanced_sql_query = """
WITH CustomerRanking AS (
    SELECT
        "ID",
        "Education",
        "TotalSpending",
        RANK() OVER(PARTITION BY "Education" ORDER BY "TotalSpending" DESC) as rank_num
    FROM
        customer_data
)
SELECT
    "ID",
    "Education",
    "TotalSpending",
    rank_num
FROM
    CustomerRanking
WHERE
    rank_num <= 5;
"""

# Execute the query and load the result into a new DataFrame
top_spenders_df = pd.read_sql_query(advanced_sql_query, engine)

print("\n--- Advanced SQL Query Result: Top 5 Spenders per Education Level ---")
display(top_spenders_df)

Successfully created database engine.
Data loaded and cleaned. 'TotalSpending' feature created.
Successfully uploaded cleaned data to PostgreSQL database.

--- Advanced SQL Query Result: Top 5 Spenders per Education Level ---


,ID,Education,TotalSpending,rank_num
0,2186,2n Cycle,2257,1
1,4947,2n Cycle,2059,2
2,3139,2n Cycle,2052,3
3,4856,2n Cycle,1853,4
4,8722,2n Cycle,1853,4
5,1951,Basic,839,1
6,5043,Basic,434,2
7,8151,Basic,263,3
8,4136,Basic,119,4
9,7620,Basic,106,5


In [10]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

# Note: We are using the 'df' DataFrame that was cleaned in Phase 2.
# If you are starting a new session, you'll need to re-run the code from Phase 2 first.

# --- Step 1: Simulate Control and Treatment Groups ---
# We'll randomly assign each customer to a variant.
np.random.seed(42) # for reproducibility
df['Variant'] = np.random.choice(['Control', 'Treatment'], size=len(df), p=[0.5, 0.5])


# --- Step 2: Create a Reusable A/B Test Function ---
# Since we'll run the test multiple times, a function is efficient.
def run_ab_test(dataframe, segment_name="Overall"):
    """Runs a Chi-Squared test and prints the conclusion."""
    contingency_table = pd.crosstab(dataframe['Variant'], dataframe['Response'])
    
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    
    alpha = 0.05
    
    print(f"--- A/B Test Results for Segment: {segment_name} ---")
    print(f"P-value: {p_value:.4f}")
    
    if p_value < alpha:
        print("Conclusion: Reject the Null Hypothesis. The difference in response rate is statistically significant.\n")
    else:
        print("Conclusion: Fail to reject the Null Hypothesis. No significant difference was found.\n")


# --- Step 3: Run the A/B Test for the Overall Population ---
run_ab_test(df, segment_name="Overall Population")


# --- Step 4: Run the Segmented A/B Test ---
# Let's analyze the results for each 'Education' level.
segment_column = 'Education'
for segment_value in df[segment_column].unique():
    # Filter the DataFrame for only the current segment
    segment_df = df[df[segment_column] == segment_value]
    run_ab_test(segment_df, segment_name=segment_value)

--- A/B Test Results for Segment: Overall Population ---
P-value: 0.5558
Conclusion: Fail to reject the Null Hypothesis. No significant difference was found.

--- A/B Test Results for Segment: Graduation ---
P-value: 0.3401
Conclusion: Fail to reject the Null Hypothesis. No significant difference was found.

--- A/B Test Results for Segment: PhD ---
P-value: 0.9120
Conclusion: Fail to reject the Null Hypothesis. No significant difference was found.

--- A/B Test Results for Segment: Master ---
P-value: 0.6578
Conclusion: Fail to reject the Null Hypothesis. No significant difference was found.

--- A/B Test Results for Segment: Basic ---
P-value: 0.5728
Conclusion: Fail to reject the Null Hypothesis. No significant difference was found.

--- A/B Test Results for Segment: 2n Cycle ---
P-value: 0.4274
Conclusion: Fail to reject the Null Hypothesis. No significant difference was found.



In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

# Note: We are using the 'df' DataFrame that was prepared in the previous phases.
# If you are starting a new session, you'll need to re-run the code from Phase 2 and 3 first.

# --- Step 1: Prepare Data for Modeling ---

# Select features (X) and target (y)
# We exclude identifiers, dates, and the columns we simulated or summed up.
features = df.drop(columns=['ID', 'Dt_Customer', 'TotalSpending', 'Variant', 'Response', 'Z_CostContact', 'Z_Revenue'])
target = df['TotalSpending']

# One-hot encode categorical features like 'Education' and 'Marital_Status'
features_encoded = pd.get_dummies(features, columns=['Education', 'Marital_Status'], drop_first=True)


# --- Step 2: Split Data into Training and Testing Sets ---
X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size=0.2, random_state=42)


# --- Step 3: Train the XGBoost Model ---
# Initialize and train the XGBoost Regressor model
xgboost_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
xgboost_model.fit(X_train, y_train)


# --- Step 4: Evaluate the Model ---
# Make predictions on the unseen test data
predictions = xgboost_model.predict(X_test)

# Calculate the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, predictions)

print("--- Model Evaluation ---")
print(f"Mean Absolute Error (MAE): ${mae:.2f}")

--- Model Evaluation ---
Mean Absolute Error (MAE): $23.68


In [15]:
# Save the final DataFrame to a CSV file for Power BI
df.to_csv('advanced_marketing_data_for_dashboard.csv', index=False)

print("Successfully saved 'advanced_marketing_data_for_dashboard.csv'.")

Successfully saved 'advanced_marketing_data_for_dashboard.csv'.


In [16]:
# Save the final DataFrame to a CSV file for Power BI
df.to_csv('advanced_marketing_data_for_dashboard.csv', index=False)

print("Successfully saved 'advanced_marketing_data_for_dashboard.csv'.")

Successfully saved 'advanced_marketing_data_for_dashboard.csv'.


In [17]:
import joblib

# After training your xgboost_model...
# Save the model to a file named 'ltv_model.pkl'
joblib.dump(xgboost_model, 'ltv_model.pkl')

['ltv_model.pkl']